# Simulation for the current APM system model and the AI-enhanced APM system model

This python script outlines a basic Agent-Based Model (ABM) developed using the Mesa framework, designed as the simulation of the current APM system model and the AI-enhanced APM system model.

Each models will run 100 times to reduce the level of error, and output the KPIs of interest.

Users can change the parameters at the beginning if needed. The original parameters are the data collected from the host company.


In [2]:
from model import OMmodel
import matplotlib.pyplot as plt
from mesa import batch_run
import mesa
import pandas as pd
import seaborn as sns
from agents import PhysicalAsset, Warehouse, Operation_crew, Maintenance_crew, Control_room, SAP,equipment_supplier

sorting = 2           # sorting time of warehouse
f1 = 700/1000000      # Failure rate from 1-5 and other component
f2 = 200/1000000
f3 = 300/1000000
f4 = 400/1000000
f5 = 500/1000000
fo = 1000/1000000
num_spareparts = 10   # number of spare parts
lead_time =3*8        # Lead time of spare parts
repair_time = 4       # Repair time needed to do maintenance
learning_time = 3     # Learning time for identifying failures
planning_time = 2
chance_to_find_failure_in_schedule = 0.9       # The chance for current APM system to find failure correcetly
maintenance_interval = 2*30*8                 # Maintenance for scheduled maintenance
repairtime_scheduledmaintenance = 1*8         # Maintenance time for scheduled maintenance
interest_rate = 0.15                          # Interest rate for financial calculation
profit_per_product = 10                        # Profit of one product
wisdom_level = 0.5                            # Original wisdom level/capbability of AI
CAPEX_AI = 500000                             # The cost to impliment AI
OPEX_AI = 100000                             # The cost every year for using AI

parameters ={"sorting":sorting,"f1":f1,"f2":f2,"f3":f3,
             "f4":f4,"f5":f5,"fo":fo,
             #lead time
             "num_spareparts":num_spareparts,"lead_time":lead_time,
             "learning_time":learning_time ,"repair_time":repair_time,
             "planning_time":planning_time,"chance_to_find_failure_in_schedule":chance_to_find_failure_in_schedule,
             "maintenance_interval":maintenance_interval,"repairtime_scheduledmaintenance":repairtime_scheduledmaintenance,
             "interest_rate":interest_rate,
             "profit_per_product":profit_per_product,
             }


model = OMmodel(seed= None,
                 #sorting time(in hours)
                 sorting=parameters['sorting'],
                 # failure rate of each critical component(in hours)
                 f1=parameters['f1'],f2=parameters['f2'],f3=parameters['f3'],
                 f4=parameters['f4'],f5=parameters['f5'],fo=parameters['fo'],
                 #number of spare parts
                 num_spareparts = parameters['num_spareparts'],
                 #number of deliver time from equipment supplier to warehouse (in hours)
                 lead_time = parameters['lead_time'],
                 #Maintenance crews's learning time for failure, repair time, planning time, (in hours) and the chance to find failures
                 learning_time = parameters['learning_time'], repair_time = parameters['repair_time'], 
                 planning_time = parameters['planning_time'], chance_to_find_failure_in_schedule = parameters['chance_to_find_failure_in_schedule'],
                 maintenance_interval = parameters['maintenance_interval'], repairtime_scheduledmaintenance = parameters['repairtime_scheduledmaintenance'],
                 interest_rate = parameters['interest_rate'],
                 profit_per_product=parameters['profit_per_product'],
                 )



n=100
param_sweep = batch_run(OMmodel, parameters, iterations=n, max_steps=10*365*8,data_collection_period=10*365*8,display_progress=True )

list=[]
for i in range(2*n):
    if i % 2 ==0:
        list.append(i)

param_sweep_df = pd.DataFrame(param_sweep)
para_2=param_sweep_df.filter(items=['NPV','MTTF','MTTR','Availability','OEE','Downtime'])
para_3=para_2.drop(index=list)
para_4=para_3.mean()
pd.set_option('display.float_format', lambda x: '%.6f' % x)
print('Time period 10 years,')
print("KPIs for the current APM of manufacturing systems of Company A are:")
print(para_4)

from modelAI import OMmodelAI
from agentsAI import PhysicalAssetAI, WarehouseAI, MaintenanceAI, UNS, EquipmentAI
parametersAI ={"sorting":sorting,"f1":f1,"f2":f2,"f3":f3,
             "f4":f4,"f5":f5,"fo":fo,
             "num_spareparts":num_spareparts,"lead_time":lead_time,
             "learning_time":learning_time ,"repair_time":repair_time,
             "planning_time":planning_time,
             
             "wisdom_level":wisdom_level,"interest_rate":interest_rate,
             "profit_per_product":profit_per_product,
             "CAPEX_AI":CAPEX_AI,"OPEX_AI":OPEX_AI}


modelAI = OMmodelAI(seed= None,
                 #sorting time(in hours)
                 sorting=parametersAI['sorting'],
                 # failure rate of each component(in hours)
                 f1=parametersAI['f1'],f2=parametersAI['f2'],f3=parametersAI['f3'],
                 f4=parametersAI['f4'],f5=parametersAI['f5'],fo=parametersAI['fo'],
                 #number of spare parts
                 num_spareparts = parametersAI['num_spareparts'],
                 #number of deliver time from equipment supplier to warehouse (in hours)
                 lead_time = parametersAI['lead_time'],
                 #Maintenance crews's learning time for failure, repair time, planning time, (in hours) and the chance to find failures
                 wisdom_level = parametersAI['wisdom_level'],
                 learning_time = parametersAI['learning_time'], repair_time = parametersAI['repair_time'], 
                 planning_time = parametersAI['planning_time'],
                 interest_rate = parametersAI['interest_rate'],profit_per_product=parametersAI['profit_per_product'],
                 CAPEX_AI=parametersAI['CAPEX_AI'],OPEX_AI=parametersAI['OPEX_AI']
                 )

m=100
AIparam_sweep = batch_run(OMmodelAI, parametersAI, iterations=m, max_steps=10*365*8,data_collection_period=10*365*8,display_progress=True )

list=[]
for j in range(2*m):
    if j % 2 ==0:
        list.append(j)

AIparam_sweep_df = pd.DataFrame(AIparam_sweep)
para_2AI=AIparam_sweep_df.filter(items=['NPV','MTTF','MTTR','Availability','OEE','Downtime'])
para_3AI=para_2AI.drop(index=list)
para_4AI=para_3AI.mean()
pd.set_option('display.float_format', lambda x: '%.6f' % x)
print('Time period 10 years,')
print("KPIs for the proposed APM of AI-based manufacturing systems of Company A are:")
print(para_4AI)

  0%|          | 0/100 [00:00<?, ?it/s]

Time period 10 years,
KPIs for the current APM of manufacturing systems of Company A are:
NPV            39913137.609348
MTTF                326.939344
MTTR                  9.943177
Availability          0.956707
OEE                   0.727097
Downtime           1264.120000
dtype: float64


  0%|          | 0/100 [00:00<?, ?it/s]

Time period 10 years,
KPIs for the proposed APM of AI-based manufacturing systems of Company A are:
NPV            46425643.698541
MTTF                995.004493
MTTR                  2.616635
Availability          0.989092
OEE                   0.863478
Downtime            318.510000
dtype: float64
